In [1]:
import numpy as np
import pandas as pd
import os
import re
#https://www.nltk.org/
from sklearn.feature_extraction.text import CountVectorizer
import tensorflow as tf
import ast
from matplotlib import pyplot as plt

In [2]:
# from keybert import KeyBERT
# kw_model = KeyBERT()

In [3]:
#Extract the dataset we are interested in, synopsis, name, and rating.
df = pd.read_csv('DataSet/animes.csv')
df = df[['title','synopsis','score']]
df.dropna(inplace=True)
df = df[~df.title.duplicated(keep='first')]
df.head()

,title,synopsis,score
0,Haikyuu!! Second Season,Following their participation at the Inter-Hig...,8.82
1,Shigatsu wa Kimi no Uso,Music accompanies the path of the human metron...,8.83
2,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,8.83
3,Fullmetal Alchemist: Brotherhood,"""In order for something to be obtained, someth...",9.23
4,Kizumonogatari III: Reiketsu-hen,After helping revive the legendary vampire Kis...,8.83


In [4]:
#Method one for generating X vectors: hash.
from sklearn.feature_extraction.text import HashingVectorizer
X = df['synopsis'].to_numpy()
y = df['score'].to_numpy()
y = np.rint(y)
print(len(y))
vectorizer = HashingVectorizer(n_features=100, norm = None, alternate_sign = False)
hashed_X = vectorizer.fit_transform(X)
#print(hashed_X)
hashed_X.shape

15192


(15192, 100)

In [5]:
#Building training sets and test sets.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(hashed_X, y, test_size = 0.2)

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
model = DecisionTreeClassifier(max_depth=100)
model.fit(X_train, y_train)
pred = model.predict(X_test)

score = accuracy_score(y_test, pred)
print("Score: " + str(score))

Score: 0.3645936163211583


Accuracy of about 35%, which is better than guessing randomly (10%), but not very good for practical predictions.

Pros of this method: Easy to implement

Cons: Not interpretable, I have no idea what type of synopsis correspond to the predictions made.

In [7]:
import nltk

#https://www.nltk.org/book/ch06.html
#https://www.datacamp.com/tutorial/text-analytics-beginners-nltk#sentiment-analysis

In [8]:
from nltk.tokenize import RegexpTokenizer
#tokenizer to remove unwanted elements from out data like symbols and numbers
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(lowercase=True,stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
text_counts= cv.fit_transform(df['synopsis'])

In [9]:
#https://splunktool.com/unknown-label-type-error-when-sklearn-naive-bayes-used-with-floating-point-numbers
score_int = df['score'].to_numpy()
score_int = np.rint(score_int)
X_train, X_test, y_train, y_test = train_test_split(text_counts, score_int, test_size=0.3, random_state=1)

In [10]:
from sklearn.naive_bayes import MultinomialNB
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
# Model Generation Using Multinomial Naive Bayes
clf = MultinomialNB().fit(X_train, y_train)
predicted= clf.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, predicted))


Accuracy: 0.48244844229925404


The accuracy is slightly higher with nltk implemented at around 47%, but there are still improvements to be made. 